In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary

In [2]:
driver = webdriver.Chrome()

url = "https://wiki.xn--rckteqa2e.com/wiki/%E3%83%9D%E3%82%B1%E3%83%A2%E3%83%B3%E3%81%AE%E5%A4%96%E5%9B%BD%E8%AA%9E%E5%90%8D%E4%B8%80%E8%A6%A7"

driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html)

In [3]:
jp2en = {}
en2jp = {}

In [4]:
tables = soup.find_all("table", class_="graytable")

for table in tables:
    for row in table.find_all("tr")[1:]:
        columns = row.find_all("td")

        jpName = columns[1].text
        enName = columns[2].text

        #末尾が改行文字になっているものがある
        if jpName[-1] == "\n":
            jpName = jpName[:-1]

        if enName[-1] == "\n":
            enName = enName[:-1]    

        jp2en[jpName] = enName
        en2jp[enName] = jpName

In [10]:
import json

In [6]:
#pokemonデータを取得

poke_data_path = "data/pokemon.json"

f = open("data/pokemon.json", "r")
data = f.read()
f.close()

pokemons = json.loads(data)

wikiの変換表と名前表記が異なるものを検知し、手動入力で修正

In [7]:
for name in pokemons:
    if name in jp2en:
        #修正不要
        continue

    #検索ページを表示
    driver.get("https://www.google.com/search?q=ポケモン+英語+%s"%(name))

    #jp2enに名前なし
    #手動修正が必要
    while(True):
        print(name+"の英語表記：")
        enName = input()

        #入力があるまで繰り返し
        #Jupyterのバグでprintが表示されない場合があるので、
        #その場合はEnter押してもう一回printできるようにするため
        if enName != "":
            break
            
    #変換辞書に追加
    jp2en[name] = enName
    en2jp[enName] = name

ガラルニャースの英語表記：
ガラルニャースの英語表記：
パルデアケンタロスの英語表記：
パルデアケンタロスの英語表記：
パルデアケンタロスの英語表記：
パルデアケンタロス(みず)の英語表記：
パルデアケンタロス(みず)の英語表記：
パルデアケンタロス(ほのお)の英語表記：
パルデアケンタロス(ほのお)の英語表記：
パルデアウパーの英語表記：
パルデアウパーの英語表記：
ヒートロトムの英語表記：
ヒートロトムの英語表記：
ウォッシュロトムの英語表記：
ウォッシュロトムの英語表記：
フロストロトムの英語表記：
フロストロトムの英語表記：
スピンロトムの英語表記：
スピンロトムの英語表記：
カットロトムの英語表記：
カットロトムの英語表記：
ルガルガン(まひる)の英語表記：
ルガルガン(まひる)の英語表記：
ルガルガン(まよなか)の英語表記：
ルガルガン(まよなか)の英語表記：
ルガルガン(たそがれ)の英語表記：
ルガルガン(たそがれ)の英語表記：
イエッサン(オス)の英語表記：
イエッサン(オス)の英語表記：
イエッサン(メス)の英語表記：
イエッサン(メス)の英語表記：


## 保存

下記の文字はillegal multibyte sequenceとなるので、変換

In [21]:
#辞書のキーを変更する関数
def change_key(dict, old_key, new_key):
    dict[new_key] = dict.pop(old_key, None)

#jp2enとjp2enをまとめて変更
def edit(old_en, new_en):
    change_key(en2jp, old_en, new_en)
    jp2en[en2jp[new_en]] = new_en

old_en2jp = en2jp.copy()
for en in old_en2jp:
    if "é" in en:
        edit(en, en.replace("é", "e"))
    if "ó" in en:
        edit(en, en.replace("ó", "o"))

In [22]:
tf = open("data/pokemon-jp2en.json", "w", encoding="ansi")
json.dump(jp2en, tf, ensure_ascii=False)
tf.close()

tf = open("data/pokemon-en2jp.json", "w", encoding="ansi")
json.dump(en2jp, tf, ensure_ascii=False)
tf.close()